Task 1: Correspondence

In [2]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import dlib

def correspondence (img):
    image = cv2.imread(img)
    image = cv2.resize(image, (900, 900))

    # find some facial landmarks on source.jpg
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect the face
    rects = detector(gray, 1)


    for rect in rects:
        # Get the landmark points
        shape = predictor(gray, rect)
        
        # Convert it to the NumPy Array
        shape_np = np.zeros((68, 2), dtype="int")
        for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)
        shape = shape_np

        # Display the landmarks
        for i, (x, y) in enumerate(shape):
        # Draw the circle to mark the keypoint 
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)

    # Display the image
    cv2.imshow("Correspondence", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

s_corr = correspondence("image/source.jpg")
d_corr = correspondence("image/dest.jpg")

Source image facial points

![Alt text](image/corr_source.png)

Destination image facial points

![Alt text](image/corr_dest.png)

Task 2: Mesh

In [5]:
import numpy as np
import cv2
import matplotlib.image as mpimg
import dlib

def triangulate (img):
    image = cv2.imread(img)
    image = cv2.resize(image, (900, 900))

    # find some facial landmarks on source.jpg
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect the face
    rects = detector(gray, 1)


    for rect in rects:
        # Get the landmark points
        shape = predictor(gray, rect)
        
        # Convert it to the NumPy Array
        shape_np = np.zeros((68, 2), dtype="int")
        for i in range(0, 68):
            shape_np[i] = (shape.part(i).x, shape.part(i).y)
        shape = shape_np

        # Display the landmarks
        for i, (x, y) in enumerate(shape):
        # Draw the circle to mark the keypoint 
            cv2.circle(image, (x, y), 1, (0, 0, 255), -1)

        # compute a trinagulation with the shape points
        # Find convex hull
        hull = cv2.convexHull(shape)

        # Draw contours

        # Find delanauy traingulation for convex hull points
        sizeImg = image.shape    
        rect = (0, 0, sizeImg[1], sizeImg[0])
        dt = cv2.Subdiv2D(rect);


        for pt in shape:
            dt.insert((int(pt[0]), int(pt[1])))
        triangles = dt.getTriangleList()
        triangles = np.array(triangles, dtype=np.int32)

        # Draw delaunay triangles
        for t in triangles:
            pt1 = (t[0], t[1])
            pt2 = (t[2], t[3])
            pt3 = (t[4], t[5])
            cv2.line(image, pt1, pt2, (0, 0, 255))
            cv2.line(image, pt2, pt3, (0, 0, 255))
            cv2.line(image, pt1, pt3, (0, 0, 255))
        
        pt1 = (t[0], t[1])
        pt2 = (t[2], t[3])
        pt3 = (t[4], t[5])
        cv2.line(image, pt1, pt2, (0, 0, 255))
        cv2.line(image, pt2, pt3, (0, 0, 255))
        cv2.line(image, pt1, pt3, (0, 0, 255))
            
    # Display the image
    cv2.imshow("Mesh", image)

    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return triangles, shape

s_triangle = triangulate("image/source.jpg")
d_triangle = triangulate("image/dest.jpg")

Source image mesh

![Alt text](image/src_tri.png)

Destination image mesh

![Alt text](image/dest_tri.png)

Task 3: Blending with a mesh

In [6]:
# To match s_tri_coord to d_tri_coord, 
# s_tri_coord -> s_tri_index, 
# Use the s_tri_index to find the corresponding d_tri_coord

# convert all the coordinates in the triangles into the index of the points in the shape
def convert_coord_index(triangle, shape):
    shape_index = np.zeros((len(triangle), 3), dtype="int")
    for i, (x, y) in enumerate(shape):
        for j, (x1, y1, x2, y2, x3, y3) in enumerate(triangle):
            if (x == x1 and y == y1):
                shape_index[j][0] = i
            elif (x == x2 and y == y2):
                shape_index[j][1] = i
            elif (x == x3 and y == y3):
                shape_index[j][2] = i
    return shape_index

# convert the s_triangle index to the d_triangle coordinates
def convert_index_coord(d_triangle_index, converted_triangle):
    d_triangle_coord = np.zeros((len(converted_triangle), 6), dtype="int")
    for i, (x, y) in enumerate(d_triangle_index):

        for j, (xy1, xy2, xy3) in enumerate(converted_triangle):

            if (i == xy1):
                d_triangle_coord[j][0] = x
                d_triangle_coord[j][1] = y
            elif (i == xy2):
                d_triangle_coord[j][2] = x
                d_triangle_coord[j][3] = y
            elif (i == xy3):
                d_triangle_coord[j][4] = x
                d_triangle_coord[j][5] = y

    return d_triangle_coord
    
covnverted_triangles = convert_coord_index(s_triangle[0], s_triangle[1])
d_triangle_matched = convert_index_coord(d_triangle[1], covnverted_triangles)

# get the inbetween coordinates between s_triangle and d_triangle
def in_between_coord(s_triangle, d_triangle, n, m):
    in_between = np.zeros((len(s_triangle), 6), dtype="int64")
    for i in range(len(s_triangle)):
        in_between[i][0] = s_triangle[i][0] + (d_triangle[i][0] - s_triangle[i][0]) * (n/m)
        in_between[i][1] = s_triangle[i][1] + (d_triangle[i][1] - s_triangle[i][1]) * (n/m)
        in_between[i][2] = s_triangle[i][2] + (d_triangle[i][2] - s_triangle[i][2]) * (n/m)
        in_between[i][3] = s_triangle[i][3] + (d_triangle[i][3] - s_triangle[i][3]) * (n/m)
        in_between[i][4] = s_triangle[i][4] + (d_triangle[i][4] - s_triangle[i][4]) * (n/m)
        in_between[i][5] = s_triangle[i][5] + (d_triangle[i][5] - s_triangle[i][5]) * (n/m)

    return in_between
    
# find all pixel point in image between a triangle
def find_point_in_triangle(triangle, image):
    # triangle = [x1, y1, x2, y2, x3, y3]
    # image = image
    # return a list of pixel point in the triangle

    # find the max and min of the triangle
    x_min = min(triangle[0], triangle[2], triangle[4])
    x_max = max(triangle[0], triangle[2], triangle[4])
    y_min = min(triangle[1], triangle[3], triangle[5])
    y_max = max(triangle[1], triangle[3], triangle[5])

    # find the point in the triangle
    point_in_triangle = []
    for x in range(x_min, x_max):
        for y in range(y_min, y_max):
            if (is_in_triangle(x, y, triangle)):
                point_in_triangle.append([x, y])

    return point_in_triangle    

# check if a point is in a triangle
def is_in_triangle(x, y, triangle):
    # x, y = point
    # triangle = [x1, y1, x2, y2, x3, y3]
    # return true if the point is in the triangle, false otherwise

    # find the area of the triangle
    area = abs((triangle[0] * (triangle[3] - triangle[5]) + triangle[2] * (triangle[5] - triangle[1]) + triangle[4] * (triangle[1] - triangle[3])) / 2)

    # find the area of the triangle with the point
    area1 = abs((x * (triangle[3] - triangle[5]) + triangle[2] * (triangle[5] - y) + triangle[4] * (y - triangle[3])) / 2)
    area2 = abs((triangle[0] * (y - triangle[5]) + x * (triangle[5] - triangle[1]) + triangle[4] * (triangle[1] - y)) / 2)
    area3 = abs((triangle[0] * (triangle[3] - y) + triangle[2] * (y - triangle[1]) + x * (triangle[1] - triangle[3])) / 2)

    # check if the sum of the area is equal to the area of the triangle
    if (area == area1 + area2 + area3):
        return True
    else:
        return False

def solve_affine_matrix(s_point, d_point):

    # s_point = [x1, y1, x2, y2, x3, y3]
    # d_point = [x1, y1, x2, y2, x3, y3]

    A = [[s_point[0], s_point[1], 1, 0, 0, 0],
         [0,0,0,s_point[0], s_point[1], 1], 
         [s_point[2], s_point[3], 1, 0, 0, 0], 
         [0,0,0,s_point[2], s_point[3], 1], 
         [s_point[4], s_point[5], 1, 0, 0, 0], 
         [0,0,0,s_point[4], s_point[5], 1]]

    A = np.array(A, dtype="float32")
    b = np.array(d_point, dtype="float32")

    C = np.linalg.solve(A, b)
    d = [[C[0], C[1], C[2]],
         [C[3], C[4], C[5]],
         [0, 0, 1]]
    
    affine_m = np.array(d, dtype="float32")
    return affine_m

def generate_affine_matrix(s_triangle, d_triangle):
    # s_triangle = [x1, y1, x2, y2, x3, y3]
    # d_triangle = [x1, y1, x2, y2, x3, y3]
    # return a list of affine matrix

    affine_matrix = []
    for i in range(len(s_triangle)):
        affine_matrix.append(solve_affine_matrix(s_triangle[i], d_triangle[i]))

    return affine_matrix


# bilinear interpolation for a point in the image returning the color of the point
def bilinear_interpolation(img, x, y):
    # img = image
    # x, y = point

    # find the color of the point
    x1 = int(x)
    x2 = x1 + 1
    y1 = int(y)
    y2 = y1 + 1

    # find the color of the 4 point
    q11 = img[y1][x1]
    q12 = img[y2][x1]
    q21 = img[y1][x2]
    q22 = img[y2][x2]


    # find the new color
    new_color = ((x2 - x) * (y2 - y) * q11) + ((x2 - x) * (y - y1) * q12) + ((x - x1) * (y2 - y) * q21) + ((x - x1) * (y - y1) * q22)

    return list(map(round, new_color))

# number of in between image
n = 50

# in between image index
m = 25

# weight of the in between image
weight_s = (n - m) / n
weight_d = m / n

dest_img = cv2.imread("image/dest.jpg")
dest_img = cv2.resize(dest_img, dsize=(900, 900), interpolation=cv2.INTER_CUBIC)
src_img = cv2.imread("image/source.jpg")
src_img = cv2.resize(src_img, dsize=(900, 900), interpolation=cv2.INTER_CUBIC)

in_between_triangles = in_between_coord(s_triangle[0], d_triangle_matched, m, n)

s_to_between = generate_affine_matrix(s_triangle[0], in_between_triangles)
d_to_between = generate_affine_matrix(d_triangle_matched, in_between_triangles)

in_between_img = np.zeros((900, 900, 3), dtype="uint8")

for i in range(len(in_between_triangles)):
    points = find_point_in_triangle(in_between_triangles[i], in_between_img)
    for j in range(len(points)):
        point = np.array([points[j][0], points[j][1], 1])
        inv_s_to_between = np.linalg.pinv(s_to_between[i])
        inv_d_to_between = np.linalg.pinv(d_to_between[i])

        s_point = inv_s_to_between.dot(point)
        d_point = inv_d_to_between.dot(point)

        rounded_s_point = list(np.around(np.array(s_point), 2))
        int_s_point = [int(rounded_s_point[0]), int(rounded_s_point[1])]
        srgb = bilinear_interpolation(src_img, int_s_point[0], int_s_point[1])

        rounded_d_point = list(np.around(np.array(d_point), 2))
        int_d_point = [int(rounded_d_point[0]), int(rounded_d_point[1])]
        drgb = bilinear_interpolation(dest_img, int_d_point[0], int_d_point[1])
        #print(point[0], point[1])
        # set the color of the point to the average of the two colors of the points
        in_between_img[point[0]][point[1]][0] = int((weight_s * srgb[0] + weight_d * drgb[0]))
        in_between_img[point[0]][point[1]][1] = int((weight_s * srgb[1] + weight_d * drgb[1]))
        in_between_img[point[0]][point[1]][2] = int((weight_s * srgb[2] + weight_d * drgb[2]))
        

cv2.imwrite('image/color_img.jpg', np.rot90(in_between_img, 3))
image_show = cv2.rotate(in_between_img, cv2.ROTATE_90_CLOCKWISE)
cv2.imshow("image", image_show)

cv2.waitKey(0)
cv2.destroyAllWindows()


50% image blend

![Alt text](image/50%25.png)

70% image blend

![Alt text](image/70%25.png)

30% image blend

![Alt text](image/30%25.png)